In [1]:
#%env CUDA_VISIBLE_DEVICES=1

import os
import re
import sys
import json
sys.path.append('./modules/multi_summ')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from collections import defaultdict

In [2]:
%%time
from modules.scouter_handler import ScouterHandler
from modules.scouter.config import default_addr, default_newspaper_idx, default_timeline_idx
from modules.topic_generator import TopicGenerator
from modules.scouter import connectES as es

from modules.multi_doc_summary import MultiDocSummary
from modules.timeline_summary import TimelineSummary
from modules.image_selector import ImageSelectionModule
from modules.outlook import ForecastSentence

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Using TensorFlow backend.


CPU times: user 10.8 s, sys: 3.49 s, total: 14.2 s
Wall time: 34.6 s


In [3]:
%%time
scouter = ScouterHandler(addr=default_addr, size=1000)
multi_doc_summarizer = MultiDocSummary("", "")
imageSelector = ImageSelectionModule("")
timeline_summ = TimelineSummary("", "")
forecast = ForecastSentence("", "")

[2019-12-24 07:17:51,923 WARNING] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



[2019-12-24 07:18:01,031 WARNING] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



weight_best loaded



[2019-12-24 07:18:02,160 WARNING] From /workspace/wise_reporter/modules/image_selector.py:14: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

[2019-12-24 07:18:02,162 INFO] Using /tmp/tfhub_modules to cache modules.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


[2019-12-24 07:18:05,676 INFO] Saver not created because there are no variables in the graph to restore


[2019-12-24 07:18:06,046 WARNING] From /workspace/wise_reporter/modules/image_selector.py:17: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.



graph defined
CPU times: user 8.85 s, sys: 1.72 s, total: 10.6 s
Wall time: 22.2 s


In [6]:
query_body = scouter.make_keyword_query_body("중국 미국")

Query : 중국 미국


In [7]:
query_body

{'query': {'bool': {'should': [{'match': {'extContent': '중국'}},
    {'match': {'extContent': '미국'}}]}}}

In [ ]:
scouter.search(query_body, data_type="newspaper", max_doc_n)

In [4]:
monthly_cluster = es.keywordGetter("2019-05-01", index='monthly_topic')

[2019-12-24 07:18:06,100 INFO] GET http://155.230.34.145:9200/monthly_topic/_search [status:200 request:0.042s]


In [5]:
for cluster in monthly_cluster['clusters']:
    print(cluster['keywords'])
    for doc_info in cluster['news_source']:
        print(doc_info['newsTitle'])
    print()

['미국', '중국', '무역']
中 협상단 오고 있는데 트럼프  中이 무역협상 파기해 (상보)
日銀 총재  미중 무역전쟁 세계경제에 광범위한 타격  경고
中상무부  美, 10일 관세 인상하면 우리도 보복 조치
WSJ  中, 무역협상서 먼저 강경태도 전환 美경제 얕봐
韓 금융시장  출렁  주가 급락 환율 28개월만에 최고치
중국이 협상 깼다  트럼프 한마디에 亞증시
또 다시 美 中 무역전쟁 亞증시 일제히 내리막
美, 협상 앞두고  대중 관세 인상  관보에 공지 中  대응조치할 것

['미국', '이란', '제재', '재개']
이란, 美 제재 재개후 원유 수출 끊겨
이란 석유장관  미국, 제재로 OPEC의 단합 흔들어
美, 이란과 거래하는 기업 금융기관 제재도 검토
美-이란 대립 악화로 중동 지정학적 위기 확산 유가 급등할까
美, 이란 제재 강화 허용된 핵활동 마저 제재 대상에 포함
폼페이오  美 이란 긴장 불구 원유시장 공급 잘 될 것
中, 미국과의  무역 긴장 가운데 이란 지지 행보(종합)

['우리나라', '외환보유액', '감소']
불황형 흑자 의 그늘 경상수지 흑자 27분기 만에 최저
1분기 경상수지 흑자 112.5억달러 2012년 이후 최소
경상수지 83개월 연속 흑자 끊기나 외국인 배당금이 좌우
경상수지 83개월 흑자행진에도...규모는 7년 만에 최소
수출 부진  1분기 경상흑자 112.5억달러...6년9개월만에  최소
수출 부진에 경상수지  휘청  6년9개월만 흑자 최소(종합)
1분기 경상수지 6년9개월 만에 최저 수출 부진 속  불황형 흑자
수출 부진  쪼그라든 경상흑자 4월 적자 전환하나?

['미국', '금리동결', 'FOMC']
美 연준  금리 동결  소식에 호주 하락
저물가 일시적  금리인하 일축한 파월.. 금융시장 출렁
美연준  당분간 금리동결  금리인하론 일축
美 연준 FOMC 정책성명  물가 목표치 하회
트럼프 말이 맞나  美 1분기 성장률 이어 4월 고용도 호조
美연준  당분간 금리동결 인내정책기조 유지  의사록 공개
美연준, 기준금

In [33]:
date = '2019-05-07'
daily_cluster = es.keywordGetter(date, index='keyword_news')

for cluster in daily_cluster['clusters']:
    print(cluster['keywords'])
    for doc_info in cluster['news_source']:
        print(doc_info['newsTitle'])
        print(doc_info['postingDate'])
        break
    print()

[2019-12-02 06:37:08,529 INFO] GET http://155.230.34.145:9200/keyword_news/_search [status:200 request:0.016s]


['중국', '트럼프', '관세']
中외교부  美와 타협 희망, 관세로는 문제 해결 못해
2019-05-07

['우리나라', '외환보유액', '감소']
불황형 흑자 의 그늘 경상수지 흑자 27분기 만에 최저
2019-05-08

['SK', '엠플러스', '계약']
엠플러스, 70억 2차전지 제조설비 공급.. 해외수주 증가
2019-05-08

['하나은행', '한부모가족', '전세자금대출']
하나은행-주금공, 한부모가족 전세대출 요건완화 금리인하
2019-05-07

['코스피지수', '하락', '내림새']
코스피 하락 마감 2170대  뒷걸음
2019-05-07



In [34]:
date = '2019-05-08'
daily_cluster = es.keywordGetter(date, index='keyword_news')

for cluster in daily_cluster['clusters']:
    print(cluster['keywords'])
    for doc_info in cluster['news_source']:
        print(doc_info['newsTitle'])
        print(doc_info['postingDate'])
        break
    print()

[2019-12-02 06:37:08,645 INFO] GET http://155.230.34.145:9200/keyword_news/_search [status:200 request:0.015s]


['수출', '부진', '경상흑자']
불황형 흑자 의 그늘 경상수지 흑자 27분기 만에 최저
2019-05-08

['트럼프', '시진핑', '관세']
中언론  美과 무역협상 어려움 대처할 자신있다
2019-05-08

['EU', '유로존', '하락']
미중 무역협상  노 딜  우려, 亞증시 일제 하락
2019-05-08

['주택', '공사', 'MBS']
개인도 주택금융공사 발행 MBS 투자 가능해진다
2019-05-08

['브렉시트', '메이', '총리', '英']
英 중앙은행, 브렉시트 연기에 기준금리 0.75% 동결
2019-05-02



In [35]:
date = '2019-05-09'
daily_cluster = es.keywordGetter(date, index='keyword_news')

for cluster in daily_cluster['clusters']:
    print(cluster['keywords'])
    for doc_info in cluster['news_source']:
        print(doc_info['newsTitle'])
        print(doc_info['postingDate'])
        break
    print()

[2019-12-02 06:37:17,842 INFO] GET http://155.230.34.145:9200/keyword_news/_search [status:200 request:0.016s]


['미국', '중국', '무역']
中 협상단 오고 있는데 트럼프  中이 무역협상 파기해 (상보)
2019-05-09

['중고차', '대출', '제한']
중고차 대출한도 시세 대비 110% 이내로 과다대출 방지한다
2019-05-09

['유니온페이', '해외', '결제']
비씨카드  이제 중국에서 모바일로 결제하세요
2019-05-09

['한은', '통화신용정책', '보고서']
한은, 새로운 금융상황지수 도입... 2017년 4분기 이후 완화기조 지속
2019-05-09

['삼성자산운용', '연금', 'TDF']
삼성자산운용, 개인연금 펀드 수탁고 1조원 돌파
2019-05-09



In [5]:
%%time
date_list = ['2019-01', '2019-02', '2019-03', '2019-04', '2019-05']

main_json = defaultdict(list)

# Graph_data
main_json['graph_topic'].append({"date":"2019-01-03", "text":"바닥 모르던 국제유가 새해 첫거래일엔 상승 마감", "hot":"True"})
main_json['graph_topic'].append({"date":"2019-02-12", "text":"사요미, MSCI지수 편입...새 지수 28일 장 마감 후 적용", "hot":"True"})
main_json['graph_topic'].append({"date":"2019-03-28", "text":"유가, 美 재고 감소에 강세 WTI 60달러 눈앞", "hot":"True"})
main_json['graph_topic'].append({"date":"2019-05-09", "text":"中 협상단 오고 있는데 트럼프  中이 무역협상 파기해", "hot":"True"})

main_json['graph_topic'].append({"date":"2019-05-07", "text":"지난달 외환보유액 4040억달러 달러화 강세로 감소"})
main_json['graph_topic'].append({"date":"2019-05-08", "text":"중앙은행, 브렉시트 연기에 기준금리 0.75% 감소"})

for date in date_list:
    print(date)
    monthly_cluster = es.keywordGetter(date+"-01", index='monthly_topic')

    cluster_info = []
    content_list = []
    for c_idx, cluster in enumerate(monthly_cluster['clusters'][:-1]):
        contents = []
        keywords = cluster['keywords']
        if c_idx == 0:
            content_list = [doc['extContent'] for doc in cluster['news_source']]
            multi_summ = multi_doc_summarizer.process_data(content_list)
            
            doc = cluster['news_source'][0]
            
            image_save_path = "./results/images/{}.jpg".format(date)
            image_avail = imageSelector.process_data(doc['newsTitle'], image_save_path, download_limit=50)
            
            doc_info = {'title':doc['newsTitle'],
                        'text':multi_summ,
                        'photo':image_save_path if image_avail else None,
                        'date':doc['postingDate']}    
        else:
            doc = cluster['news_source'][0]
                        
            doc_info = {'title':doc['newsTitle'],
                        'date':doc['postingDate']}
            
        main_json[date].append(doc_info)
        
with open("./results/main.json", "w", encoding='utf8') as fp:
    fp.write(json.dumps(main_json, ensure_ascii=False))

[2019-12-09 07:37:41,407 INFO] GET http://155.230.34.145:9200/monthly_topic/_search [status:200 request:0.013s]


2019-01


[2019-12-09 07:38:04,953 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.011s]


1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : others
4.jpg : , Predict : others
5.jpg : , Predict : others
6.jpg : , Predict : others
7.jpg : , Predict : others
8.jpg : , Predict : others
9.jpg : , Predict : others
10.jpg : , Predict : others
11.jpg : , Predict : others
12.jpg : , Predict : others
13.jpg : , Predict : others
14.jpg : , Predict : graph
15.jpg : , Predict : others
16.jpg : , Predict : others
17.jpg : , Predict : graph
18.jpg : , Predict : others
19.jpg : , Predict : others
20.jpg : , Predict : others
21.jpg : , Predict : others
22.jpg : , Predict : others
23.jpg : , Predict : others
24.jpg : , Predict : others
25.jpg : , Predict : graph
26.jpg : , Predict : others
27.jpg : , Predict : others
28.jpg : , Predict : others
29.jpg : , Predict : others
30.jpg : , Predict : others
31.jpg : , Predict : others
32.jpg : , Predict : others
33.jpg : , Predict : graph
34.jpg : , Predict : others
35.jpg : , Predict : others
36.jpg : , Predict : others
37.jp

[2019-12-09 07:38:09,202 INFO] GET http://155.230.34.145:9200/monthly_topic/_search [status:200 request:0.017s]


...

1.jpg : , Predict : graph
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : graph
10.jpg : , Predict : graph
11.jpg : , Predict : graph
12.jpg : , Predict : graph
13.jpg : , Predict : graph
14.jpg : , Predict : graph
15.jpg : , Predict : others
16.jpg : , Predict : graph
17.jpg : , Predict : graph
18.jpg : , Predict : graph
19.jpg : , Predict : graph
20.jpg : , Predict : graph
21.jpg : , Predict : graph
22.jpg : , Predict : others
23.jpg : , Predict : graph
24.jpg : , Predict : others
25.jpg : , Predict : graph
26.jpg : , Predict : graph
27.jpg : , Predict : others
28.jpg : , Predict : graph
29.jpg : , Predict : graph
30.jpg : , Predict : graph
31.jpg : , Predict : graph
32.jpg : , Predict : graph
33.jpg : , Predict : graph
34.jpg : , Predict : graph
35.jpg : , Predict : others
36.jpg : , Predict : graph
37.jpg : , Predict : graph
38.jp

[2019-12-09 07:38:41,034 INFO] GET http://155.230.34.145:9200/monthly_topic/_search [status:200 request:0.016s]


2019-05


[2019-12-09 07:38:55,185 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.010s]


1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : others
4.jpg : , Predict : others
5.jpg : , Predict : others
6.jpg : , Predict : others
7.jpg : , Predict : others
8.jpg : , Predict : others
9.jpg : , Predict : others
10.jpg : , Predict : others
11.jpg : , Predict : others
12.jpg : , Predict : others
13.jpg : , Predict : others
14.jpg : , Predict : others
15.jpg : , Predict : graph
16.jpg : , Predict : others
17.jpg : , Predict : others
18.jpg : , Predict : others
19.jpg : , Predict : others
20.jpg : , Predict : others
21.jpg : , Predict : others
22.jpg : , Predict : others
23.jpg : , Predict : graph
24.jpg : , Predict : others
25.jpg : , Predict : others
26.jpg : , Predict : others
27.jpg : , Predict : others
28.jpg : , Predict : others
29.jpg : , Predict : others
30.jpg : , Predict : others
31.jpg : , Predict : graph
32.jpg : , Predict : graph
33.jpg : , Predict : others
34.jpg : , Predict : others
35.jpg : , Predict : graph
36.jpg : , Predict : others
37.jpg

In [5]:
main_json.keys()

dict_keys(['2019-01', '2019-02', '2019-03', '2019-04', '2019-05'])

In [5]:
%%time
date = '2019-03-28'
monthly_cluster = es.keywordGetter(date, index='keyword_news')

daily_json = []
keyword_list = []
for c_idx, cluster in enumerate(monthly_cluster['clusters']):
    print(cluster['keywords'])
    keyword_list.append(cluster['keywords'])
    
    content_list = [doc['extContent'] for doc in cluster['news_source']]
    multi_summ = multi_doc_summarizer.process_data(content_list)
    
    doc = cluster['news_source'][0]
    
    image_save_path = "./results/images/{}_{}.jpg".format(date, c_idx)
    image_avail = imageSelector.process_data(doc['newsTitle'], image_save_path, download_limit=50)
    
    doc_info = {'title':doc['newsTitle'],
                'keyword':cluster['keywords'],
                'text':multi_summ,
                'photo':image_save_path if image_avail else None}
    
    daily_json.append(doc_info)
    
with open("./results/daily_issue/{}.json".format(date), "w", encoding='utf8') as fp:
    fp.write(json.dumps(daily_json, ensure_ascii=False))

[2019-12-10 02:09:58,963 INFO] GET http://155.230.34.145:9200/keyword_news/_search [status:200 request:0.015s]


['터키', '리라', '급락']


[2019-12-10 02:10:08,159 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.011s]


1.jpg : , Predict : graph
2.jpg : , Predict : others
3.jpg : , Predict : others
4.jpg : , Predict : others
5.jpg : , Predict : others
6.jpg : , Predict : others
7.jpg : , Predict : others
8.jpg : , Predict : others
9.jpg : , Predict : others
10.jpg : , Predict : others
11.jpg : , Predict : others
12.jpg : , Predict : others
13.jpg : , Predict : others
14.jpg : , Predict : graph
15.jpg : , Predict : others
16.jpg : , Predict : others
17.jpg : , Predict : others
18.jpg : , Predict : others
19.jpg : , Predict : graph
20.jpg : , Predict : graph
21.jpg : , Predict : graph
22.jpg : , Predict : others
23.jpg : , Predict : graph
24.jpg : , Predict : others
25.jpg : , Predict : others
26.jpg : , Predict : others
27.jpg : , Predict : others
28.jpg : , Predict : others
29.jpg : , Predict : graph
30.jpg : , Predict : others
31.jpg : , Predict : others
32.jpg : , Predict : graph
33.jpg : , Predict : others
34.jpg : , Predict : graph
35.jpg : , Predict : others
36.jpg : , Predict : graph
37.jpg : , 

[2019-12-10 02:10:33,805 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.012s]


...

1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : others
4.jpg : , Predict : others
5.jpg : , Predict : others
6.jpg : , Predict : others
7.jpg : , Predict : others
8.jpg : , Predict : others
9.jpg : , Predict : others
10.jpg : , Predict : others
11.jpg : , Predict : others
12.jpg : , Predict : others
13.jpg : , Predict : others
14.jpg : , Predict : others
15.jpg : , Predict : others
16.jpg : , Predict : others
17.jpg : , Predict : others
18.jpg : , Predict : others
19.jpg : , Predict : others
20.jpg : , Predict : others
21.jpg : , Predict : graph
22.jpg : , Predict : others
23.jpg : , Predict : others
24.jpg : , Predict : others
25.jpg : , Predict : others
26.jpg : , Predict : others
27.jpg : , Predict : others
28.jpg : , Predict : others
29.jpg : , Predict : others
30.jpg : , Predict : others
31.jpg : , Predict : others
32.jpg : , Predict : others
33.jpg : , Predict : others
34.jpg : , Predict : others
35.jpg : , Predict : others
36.jpg : , Predict : others
37

[2019-12-10 02:11:13,456 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.013s]


1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : others
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : others
9.jpg : , Predict : graph
10.jpg : , Predict : graph
11.jpg : , Predict : graph
12.jpg : , Predict : graph
13.jpg : , Predict : graph
14.jpg : , Predict : graph
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : graph
18.jpg : , Predict : graph
19.jpg : , Predict : graph
20.jpg : , Predict : graph
21.jpg : , Predict : graph
22.jpg : , Predict : graph
23.jpg : , Predict : graph
24.jpg : , Predict : graph
25.jpg : , Predict : graph
26.jpg : , Predict : graph
27.jpg : , Predict : others
28.jpg : , Predict : graph
29.jpg : , Predict : graph
30.jpg : , Predict : graph
31.jpg : , Predict : graph
32.jpg : , Predict : graph
33.jpg : , Predict : graph
34.jpg : , Predict : graph
35.jpg : , Predict : others
36.jpg : , Predict : graph
37.jpg : , Predict : graph
38.j

[2019-12-10 02:11:33,296 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.014s]


1.jpg : , Predict : graph
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : graph
10.jpg : , Predict : graph
11.jpg : , Predict : graph
12.jpg : , Predict : graph
13.jpg : , Predict : others
14.jpg : , Predict : others
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : graph
18.jpg : , Predict : graph
19.jpg : , Predict : graph
20.jpg : , Predict : graph
21.jpg : , Predict : graph
22.jpg : , Predict : graph
23.jpg : , Predict : graph
24.jpg : , Predict : graph
25.jpg : , Predict : graph
26.jpg : , Predict : others
27.jpg : , Predict : graph
28.jpg : , Predict : graph
29.jpg : , Predict : graph
30.jpg : , Predict : graph
31.jpg : , Predict : graph
32.jpg : , Predict : graph
33.jpg : , Predict : graph
34.jpg : , Predict : graph
35.jpg : , Predict : graph
36.jpg : , Predict : graph
37.jpg : , Predict : graph
38.jpg 

In [4]:
%%time
date = '2019-05-07'
monthly_cluster = es.keywordGetter(date, index='keyword_news')

daily_json = []
keyword_list = []
for c_idx, cluster in enumerate(monthly_cluster['clusters']):
    print(cluster['keywords'])
    keyword_list.append(cluster['keywords'])
    
    content_list = [doc['extContent'] for doc in cluster['news_source']]
    multi_summ = multi_doc_summarizer.process_data(content_list)
    
    doc = cluster['news_source'][0]
    
    image_save_path = "./results/images/{}_{}.jpg".format(date, c_idx)
    image_avail = imageSelector.process_data(doc['newsTitle'], image_save_path, download_limit=50)
    
    doc_info = {'title':doc['newsTitle'],
                'keyword':cluster['keywords'],
                'text':multi_summ,
                'photo':image_save_path if image_avail else None}
    
    daily_json.append(doc_info)
    
with open("./results/daily_issue/{}.json".format(date), "w", encoding='utf8') as fp:
    fp.write(json.dumps(daily_json, ensure_ascii=False))

[2019-12-09 07:40:08,916 INFO] GET http://155.230.34.145:9200/keyword_news/_search [status:200 request:0.053s]


['중국', '트럼프', '관세']


[2019-12-09 07:40:22,034 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.013s]


1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : others
4.jpg : , Predict : others
5.jpg : , Predict : graph
6.jpg : , Predict : others
7.jpg : , Predict : others
8.jpg : , Predict : others
9.jpg : , Predict : others
10.jpg : , Predict : others
11.jpg : , Predict : others
12.jpg : , Predict : others
13.jpg : , Predict : others
14.jpg : , Predict : others
15.jpg : , Predict : others
16.jpg : , Predict : others
17.jpg : , Predict : others
18.jpg : , Predict : others
19.jpg : , Predict : others
20.jpg : , Predict : others
21.jpg : , Predict : others
22.jpg : , Predict : others
23.jpg : , Predict : others
24.jpg : , Predict : graph
25.jpg : , Predict : others
26.jpg : , Predict : others
27.jpg : , Predict : graph
28.jpg : , Predict : others
29.jpg : , Predict : graph
30.jpg : , Predict : others
31.jpg : , Predict : others
32.jpg : , Predict : others
33.jpg : , Predict : others
34.jpg : , Predict : others
35.jpg : , Predict : graph
36.jpg : , Predict : others
37.jpg

[2019-12-09 07:40:35,053 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.010s]


...

1.jpg : , Predict : others
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : others
10.jpg : , Predict : graph
11.jpg : , Predict : graph
12.jpg : , Predict : graph
13.jpg : , Predict : others
14.jpg : , Predict : graph
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : graph
18.jpg : , Predict : others
19.jpg : , Predict : graph
20.jpg : , Predict : graph
21.jpg : , Predict : others
22.jpg : , Predict : graph
23.jpg : , Predict : graph
24.jpg : , Predict : others
25.jpg : , Predict : graph
26.jpg : , Predict : graph
27.jpg : , Predict : graph
28.jpg : , Predict : graph
29.jpg : , Predict : graph
30.jpg : , Predict : graph
31.jpg : , Predict : graph
32.jpg : , Predict : graph
33.jpg : , Predict : graph
34.jpg : , Predict : graph
35.jpg : , Predict : others
36.jpg : , Predict : graph
37.jpg : , Predict : graph
38.

[2019-12-09 07:40:51,380 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.009s]


1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : others
4.jpg : , Predict : others
5.jpg : , Predict : others
6.jpg : , Predict : others
7.jpg : , Predict : others
8.jpg : , Predict : graph
9.jpg : , Predict : others
10.jpg : , Predict : others
11.jpg : , Predict : others
12.jpg : , Predict : others
13.jpg : , Predict : graph
14.jpg : , Predict : graph
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : graph
18.jpg : , Predict : graph
19.jpg : , Predict : graph
20.jpg : , Predict : others
21.jpg : , Predict : graph
22.jpg : , Predict : graph
23.jpg : , Predict : others
24.jpg : , Predict : graph
25.jpg : , Predict : graph
26.jpg : , Predict : graph
27.jpg : , Predict : others
28.jpg : , Predict : graph
29.jpg : , Predict : graph
30.jpg : , Predict : graph
31.jpg : , Predict : graph
32.jpg : , Predict : graph
33.jpg : , Predict : others
34.jpg : , Predict : graph
35.jpg : , Predict : others
36.jpg : , Predict : others
37.jpg : , Predict :

[2019-12-09 07:40:56,622 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.009s]


1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : others
4.jpg : , Predict : others
5.jpg : , Predict : others
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : others
9.jpg : , Predict : others
10.jpg : , Predict : others
11.jpg : , Predict : others
12.jpg : , Predict : others
13.jpg : , Predict : graph
14.jpg : , Predict : others
15.jpg : , Predict : others
16.jpg : , Predict : others
17.jpg : , Predict : others
18.jpg : , Predict : others
19.jpg : , Predict : others
20.jpg : , Predict : others
21.jpg : , Predict : others
22.jpg : , Predict : graph
23.jpg : , Predict : others
24.jpg : , Predict : others
25.jpg : , Predict : others
26.jpg : , Predict : others
27.jpg : , Predict : graph
28.jpg : , Predict : others
29.jpg : , Predict : graph
30.jpg : , Predict : others
31.jpg : , Predict : others
32.jpg : , Predict : others
33.jpg : , Predict : graph
34.jpg : , Predict : others
35.jpg : , Predict : graph
36.jpg : , Predict : graph
37.jpg : ,

In [5]:
%%time
date = '2019-05-08'
monthly_cluster = es.keywordGetter(date, index='keyword_news')

daily_json = []
keyword_list = []
for c_idx, cluster in enumerate(monthly_cluster['clusters']):
    print(cluster['keywords'])
    keyword_list.append(cluster['keywords'])
    
    content_list = [doc['extContent'] for doc in cluster['news_source']]
    multi_summ = multi_doc_summarizer.process_data(content_list)
    
    doc = cluster['news_source'][0]
    
    image_save_path = "./results/images/{}_{}.jpg".format(date, c_idx)
    image_avail = imageSelector.process_data(doc['newsTitle'], image_save_path, download_limit=50)
    
    doc_info = {'title':doc['newsTitle'],
                'keyword':cluster['keywords'],
                'text':multi_summ,
                'photo':image_save_path if image_avail else None}
    
    daily_json.append(doc_info)
    
with open("./results/daily_issue/{}.json".format(date), "w", encoding='utf8') as fp:
    fp.write(json.dumps(daily_json, ensure_ascii=False))

[2019-12-09 07:41:00,829 INFO] GET http://155.230.34.145:9200/keyword_news/_search [status:200 request:0.014s]


['수출', '부진', '경상흑자']


[2019-12-09 07:41:13,368 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.010s]


1.jpg : , Predict : others
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : graph
10.jpg : , Predict : graph
11.jpg : , Predict : graph
12.jpg : , Predict : graph
13.jpg : , Predict : graph
14.jpg : , Predict : graph
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : graph
18.jpg : , Predict : graph
19.jpg : , Predict : graph
20.jpg : , Predict : graph
21.jpg : , Predict : others
22.jpg : , Predict : graph
23.jpg : , Predict : graph
24.jpg : , Predict : graph
25.jpg : , Predict : graph
26.jpg : , Predict : graph
27.jpg : , Predict : graph
28.jpg : , Predict : graph
29.jpg : , Predict : graph
30.jpg : , Predict : graph
31.jpg : , Predict : others
32.jpg : , Predict : graph
33.jpg : , Predict : graph
34.jpg : , Predict : graph
35.jpg : , Predict : graph
36.jpg : , Predict : graph
37.jpg : , Predict : others
38.jpg

[2019-12-09 07:41:22,668 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.010s]


...

1.jpg : , Predict : graph
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : others
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : graph
10.jpg : , Predict : graph
11.jpg : , Predict : graph
12.jpg : , Predict : graph
13.jpg : , Predict : graph
14.jpg : , Predict : graph
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : graph
18.jpg : , Predict : graph
19.jpg : , Predict : others
20.jpg : , Predict : graph
21.jpg : , Predict : graph
22.jpg : , Predict : others
23.jpg : , Predict : graph
24.jpg : , Predict : graph
25.jpg : , Predict : graph
26.jpg : , Predict : graph
27.jpg : , Predict : others
28.jpg : , Predict : graph
29.jpg : , Predict : others
30.jpg : , Predict : graph
31.jpg : , Predict : graph
32.jpg : , Predict : others
33.jpg : , Predict : graph
34.jpg : , Predict : graph
35.jpg : , Predict : graph
36.jpg : , Predict : others
37.jpg : , Predict : others
38

[2019-12-09 07:41:58,682 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.008s]


1.jpg : , Predict : graph
2.jpg : , Predict : graph
3.jpg : , Predict : others
4.jpg : , Predict : graph
5.jpg : , Predict : others
6.jpg : , Predict : others
7.jpg : , Predict : others
8.jpg : , Predict : graph
9.jpg : , Predict : others
10.jpg : , Predict : others
11.jpg : , Predict : others
12.jpg : , Predict : graph
13.jpg : , Predict : others
14.jpg : , Predict : graph
15.jpg : , Predict : others
16.jpg : , Predict : others
17.jpg : , Predict : graph
18.jpg : , Predict : others
19.jpg : , Predict : graph
20.jpg : , Predict : others
21.jpg : , Predict : others
22.jpg : , Predict : graph
23.jpg : , Predict : graph
24.jpg : , Predict : graph
25.jpg : , Predict : graph
26.jpg : , Predict : graph
27.jpg : , Predict : others
28.jpg : , Predict : graph
29.jpg : , Predict : graph
30.jpg : , Predict : graph
31.jpg : , Predict : graph
32.jpg : , Predict : graph
33.jpg : , Predict : graph
34.jpg : , Predict : graph
35.jpg : , Predict : others
36.jpg : , Predict : graph
37.jpg : , Predict : g

[2019-12-09 07:42:07,290 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.009s]


1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : others
4.jpg : , Predict : others
5.jpg : , Predict : others
6.jpg : , Predict : others
7.jpg : , Predict : others
8.jpg : , Predict : others
9.jpg : , Predict : others
10.jpg : , Predict : graph
11.jpg : , Predict : others
12.jpg : , Predict : graph
13.jpg : , Predict : others
14.jpg : , Predict : others
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : graph
18.jpg : , Predict : others
19.jpg : , Predict : others
20.jpg : , Predict : graph
21.jpg : , Predict : graph
22.jpg : , Predict : graph
23.jpg : , Predict : others
24.jpg : , Predict : graph
25.jpg : , Predict : others
26.jpg : , Predict : graph
27.jpg : , Predict : graph
28.jpg : , Predict : graph
29.jpg : , Predict : graph
30.jpg : , Predict : others
31.jpg : , Predict : graph
32.jpg : , Predict : graph
33.jpg : , Predict : others
34.jpg : , Predict : graph
35.jpg : , Predict : graph
36.jpg : , Predict : others
37.jpg : , Predict

In [6]:
%%time
date = '2019-05-09'
monthly_cluster = es.keywordGetter(date, index='keyword_news')

daily_json = []
keyword_list = []
for c_idx, cluster in enumerate(monthly_cluster['clusters']):
    print(cluster['keywords'])
    keyword_list.append(cluster['keywords'])
    
    content_list = [doc['extContent'] for doc in cluster['news_source']]
    multi_summ = multi_doc_summarizer.process_data(content_list)
    
    doc = cluster['news_source'][0]
    
    image_save_path = "./results/images/{}_{}.jpg".format(date, c_idx)
    image_avail = imageSelector.process_data(doc['newsTitle'], image_save_path, download_limit=50)
    
    doc_info = {'title':doc['newsTitle'],
                'keyword':cluster['keywords'],
                'text':multi_summ,
                'photo':image_save_path if image_avail else None}
    
    daily_json.append(doc_info)
    
with open("./results/daily_issue/{}.json".format(date), "w", encoding='utf8') as fp:
    fp.write(json.dumps(daily_json, ensure_ascii=False))

[2019-12-09 07:42:11,487 INFO] GET http://155.230.34.145:9200/keyword_news/_search [status:200 request:0.015s]


['미국', '중국', '무역']


[2019-12-09 07:42:25,580 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.009s]


1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : others
4.jpg : , Predict : others
5.jpg : , Predict : others
6.jpg : , Predict : others
7.jpg : , Predict : others
8.jpg : , Predict : others
9.jpg : , Predict : others
10.jpg : , Predict : others
11.jpg : , Predict : others
12.jpg : , Predict : others
13.jpg : , Predict : others
14.jpg : , Predict : others
15.jpg : , Predict : graph
16.jpg : , Predict : others
17.jpg : , Predict : others
18.jpg : , Predict : others
19.jpg : , Predict : others
20.jpg : , Predict : others
21.jpg : , Predict : others
22.jpg : , Predict : others
23.jpg : , Predict : graph
24.jpg : , Predict : others
25.jpg : , Predict : others
26.jpg : , Predict : others
27.jpg : , Predict : others
28.jpg : , Predict : others
29.jpg : , Predict : others
30.jpg : , Predict : others
31.jpg : , Predict : graph
32.jpg : , Predict : graph
33.jpg : , Predict : others
34.jpg : , Predict : others
35.jpg : , Predict : graph
36.jpg : , Predict : others
37.jpg

[2019-12-09 07:42:38,387 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.009s]


...

1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : graph
4.jpg : , Predict : others
5.jpg : , Predict : graph
6.jpg : , Predict : others
7.jpg : , Predict : others
8.jpg : , Predict : others
9.jpg : , Predict : others
10.jpg : , Predict : others
11.jpg : , Predict : others
12.jpg : , Predict : others
13.jpg : , Predict : graph
14.jpg : , Predict : others
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : others
18.jpg : , Predict : others
19.jpg : , Predict : others
20.jpg : , Predict : graph
21.jpg : , Predict : others
22.jpg : , Predict : graph
23.jpg : , Predict : graph
24.jpg : , Predict : others
25.jpg : , Predict : others
26.jpg : , Predict : others
27.jpg : , Predict : others
28.jpg : , Predict : graph
29.jpg : , Predict : others
30.jpg : , Predict : others
31.jpg : , Predict : others
32.jpg : , Predict : others
33.jpg : , Predict : others
34.jpg : , Predict : others
35.jpg : , Predict : graph
36.jpg : , Predict : graph
37.jpg : , P

[2019-12-09 07:42:56,445 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.009s]


1.jpg : , Predict : graph
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : graph
10.jpg : , Predict : graph
11.jpg : , Predict : graph
12.jpg : , Predict : graph
13.jpg : , Predict : graph
14.jpg : , Predict : graph
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : graph
18.jpg : , Predict : graph
19.jpg : , Predict : graph
20.jpg : , Predict : graph
21.jpg : , Predict : others
22.jpg : , Predict : graph
23.jpg : , Predict : graph
24.jpg : , Predict : graph
25.jpg : , Predict : graph
26.jpg : , Predict : graph
27.jpg : , Predict : graph
28.jpg : , Predict : graph
29.jpg : , Predict : graph
30.jpg : , Predict : graph
31.jpg : , Predict : graph
32.jpg : , Predict : graph
33.jpg : , Predict : graph
34.jpg : , Predict : graph
35.jpg : , Predict : others
36.jpg : , Predict : graph
37.jpg : , Predict : others
38.jpg 

[2019-12-09 07:43:10,447 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.009s]


1.jpg : , Predict : others
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : others
9.jpg : , Predict : others
10.jpg : , Predict : others
11.jpg : , Predict : others
12.jpg : , Predict : others
13.jpg : , Predict : graph
14.jpg : , Predict : graph
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : others
18.jpg : , Predict : graph
19.jpg : , Predict : graph
20.jpg : , Predict : graph
21.jpg : , Predict : graph
22.jpg : , Predict : graph
23.jpg : , Predict : others
24.jpg : , Predict : others
25.jpg : , Predict : graph
26.jpg : , Predict : graph
27.jpg : , Predict : graph
28.jpg : , Predict : graph
29.jpg : , Predict : graph
30.jpg : , Predict : graph
31.jpg : , Predict : graph
32.jpg : , Predict : graph
33.jpg : , Predict : graph
34.jpg : , Predict : others
35.jpg : , Predict : graph
36.jpg : , Predict : graph
37.jpg : , Predict : others

In [11]:
query_body = scouter.make_keyword_query_body("미국중국무역", match_item='keyword')
detail_summary_data = scouter.search(query_body, data_type="detail_topic", max_num_doc=1000, trim_lower=False)[0]

[2019-12-06 06:47:07,400 INFO] GET http://155.230.34.145:9200/detail_topic/_search?scroll=10m&size=1000 [status:200 request:0.154s]


Query : 미국중국무역
Scroll idx : 1 (1 docs)


[2019-12-06 06:47:07,453 INFO] GET http://155.230.34.145:9200/_search/scroll?scroll=10m [status:200 request:0.007s]


Total retrieved Doc # :  1



In [13]:
detail_summary_data['subkeywords']

['글로벌 경기 둔화',
 '류허 부총리',
 '관세 인상',
 '로버트 라이트하이저',
 '므누신 재무장관',
 '미국산 대두 수입',
 '환율 관찰 대상국']

daily_json

In [5]:
%%time
keyword_set_list = [
#    ['트럼프', 'OPEC', '발언'], #
    ['우리나라', '외환보유액', '감소'], #
    ['브렉시트', '메이', '총리', '英'], #
    ['미국', '중국', '무역'] #
]

for keyword_set in keyword_set_list:
    query_body = scouter.make_keyword_query_body(''.join(keyword_set), match_item='keyword')
    detail_summary_data = scouter.search(query_body, data_type="detail_topic", max_num_doc=1000, trim_lower=False)[0]

    detail_json = {}

    if keyword_set == ['트럼프', 'OPEC', '발언']:
        title = "국제 유가 변동"
        subkeywords = ["국제 유가 급등에 희비 엇갈린 조선 정유 화학 항공주",
                       "OPEC 석유감산 합의",
                       "美 베네수엘라 원유제재",
                       "원유펀드 수익률 40% 돌파",
                       "사우디 해외 원유 가스 개발"]
        print_title_image_path = "./results/images/트럼프_OPEC_발언_print_ver_title.jpg"
        
    elif keyword_set == ['우리나라', '외환보유액', '감소']:
        title = "외환보유액 감소"
        subkeywords = ["국제 유가 변동",
                       "글로벌 달러화 강세",
                       "유가증권 감소",
                       "우리회계법인 글로벌 컨설팅그룹  베이커틸리 제휴"]
        print_title_image_path = "./results/images/우리나라_외환보유액_감소_print_ver_title.jpg"
        
    elif keyword_set == ['브렉시트', '메이', '총리', '英']:
        title = "브렉시트"
        subkeywords = ["EU 관세동맹",
                       "노딜 브렉시트",
                       "브렉시트 합의안 부결",
                       "메이 총리 사임"]
        print_title_image_path = "./results/images/브렉시트_메이_총리_英_print_ver_title.jpg"
        
    elif keyword_set == ['미국', '중국', '무역']:        
        title = "미중 무역전쟁"
        subkeywords = ["글로벌 경기둔화",
                       "미중 무역전쟁 분수령 관세인상",
                       "中 미국산 대두수입 중단",
                       "美재무부, 9개국 환율관찰국 지정"]
        print_title_image_path = "./results/images/미국_중국_무역_print_ver_title.jpg"
        
    else:
        title = ' '.join(keyword_set)
        subkeywords = []
        print_title_image_path = None
        
    detail_json['print_title_photo'] = print_title_image_path
        
    subkeywords = [{"title":key, "keyword":[]} for key in subkeywords]
    
    outline_info = {"title":title,
                    "text":[],
                    "link":[]}

    for section in detail_summary_data['section']:
        outline_info['text'].append(multi_doc_summarizer.process_data([doc['extContent'] for doc in section]))
        outline_info['link'].extend([doc['originalUrl'] for doc in section])

    detail_json['outline'] = outline_info
    detail_json['subkeywords'] = subkeywords

    keyword_for_timeline = ' '.join(keyword_set) if ' '.join(keyword_set) != '트럼프 OPEC 발언' else '국제 유가 OPEC'
    query_body = scouter.make_keyword_query_body(keyword_for_timeline, match_item='extContent')
    detail_timeline_data = scouter.search(query_body, data_type=default_timeline_idx, max_num_doc=1000, trim_lower=False)
    detail_timeline_data = [doc for doc in detail_timeline_data if "error" not in doc.keys()]

    summary, _ = timeline_summ.process_data(keyword_for_timeline, detail_timeline_data)
    dates = [sentence['date'] for sentence in summary]
    timeline_sen = [' '.join(sentence['compResult']) for sentence in summary]
    normal_sentence = [sentence['article'][0]['sentence'].strip() for sentence in summary]

    detail_json['timeline'] = []
    for t_idx, (sen, date) in enumerate(zip(normal_sentence, dates)):
        print(sen)
        image_save_path = "./results/images/{}_{}.jpg".format('_'.join(keyword_set), t_idx)
        image_avail = imageSelector.process_data(sen, image_save_path, download_limit=50)
    
        timeline_section = {"text":sen,
                            "date":date,
                            "photo":image_save_path if image_avail else None}

        detail_json['timeline'].append(timeline_section)

    query_body = scouter.make_keyword_query_body(keyword_for_timeline, match_item='extContent', filters=['analyzed_text', 'postingDate'])
    detail_outlook_data = scouter.search(query_body, data_type=default_newspaper_idx, max_num_doc=500, trim_lower=False)

    forecast_sentence = forecast.process_data(detail_outlook_data, 5)

    detail_json['outlook'] = [sen.split('\t')[0].strip() for sen in forecast_sentence]

    with open("./results/detail_issue/{}.json".format('_'.join(keyword_set)), "w", encoding='utf8') as fp:
        fp.write(json.dumps(detail_json, ensure_ascii=False))

[2019-12-10 05:33:09,992 INFO] GET http://155.230.34.145:9200/detail_topic/_search?scroll=10m&size=1000 [status:200 request:0.148s]


Query : 우리나라외환보유액감소
Scroll idx : 1 (1 docs)


[2019-12-10 05:33:10,048 INFO] GET http://155.230.34.145:9200/_search/scroll?scroll=10m [status:200 request:0.006s]


Total retrieved Doc # :  1

Query : 우리나라 외환보유액 감소


[2019-12-10 05:33:48,970 INFO] GET http://155.230.34.145:9200/summary_by_time/_search?scroll=10m&size=1000 [status:200 request:1.136s]


...

중국 인민은행은 위안화 고시환율을 1달러당 6.8365위안으로 고시했다.


[2019-12-10 05:39:29,917 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.012s]


중국 인민은행은 달러 대비 위안화 중간 환율을 전 거래일보다 0.02% 오른 6.8988위안으로 고시했다.
Query : 미국 중국 무역


[2019-12-10 05:39:42,373 INFO] GET http://155.230.34.145:9200/newspaper/_search?scroll=10m&size=500 [status:200 request:4.158s]


Scroll idx : 1 (500 docs)
Total retrieved Doc # :  500

CPU times: user 4min 34s, sys: 20.1 s, total: 4min 54s
Wall time: 6min 47s


In [8]:
image_save_path = "./results/images/우리나라_외환보유액_감소_0.jpg"
image_avail = imageSelector.process_data("달러 약세로 지난달 외환보유액이 사상 최대 규모인 4,055억달러를 기록했다.", image_save_path, download_limit=50)

[2019-12-09 07:47:25,567 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.072s]


1.jpg : , Predict : others
2.jpg : , Predict : graph
3.jpg : , Predict : others
4.jpg : , Predict : others
5.jpg : , Predict : others
6.jpg : , Predict : graph
7.jpg : , Predict : others
8.jpg : , Predict : others
9.jpg : , Predict : others
10.jpg : , Predict : others
11.jpg : , Predict : others
12.jpg : , Predict : graph
13.jpg : , Predict : graph
14.jpg : , Predict : others
15.jpg : , Predict : others
16.jpg : , Predict : graph
17.jpg : , Predict : others
18.jpg : , Predict : others
19.jpg : , Predict : others
20.jpg : , Predict : graph
21.jpg : , Predict : graph
22.jpg : , Predict : graph
23.jpg : , Predict : graph
24.jpg : , Predict : graph
25.jpg : , Predict : graph
26.jpg : , Predict : graph
27.jpg : , Predict : others
28.jpg : , Predict : others
29.jpg : , Predict : graph
30.jpg : , Predict : others
31.jpg : , Predict : others
32.jpg : , Predict : others
33.jpg : , Predict : others
34.jpg : , Predict : others
35.jpg : , Predict : graph
36.jpg : , Predict : others
37.jpg : , Pred

In [9]:
image_save_path = "./results/images/우리나라_외환보유액_감소_1.jpg"
image_avail = imageSelector.process_data("외환보유액 1위 중국 3만879억달러, 2위는 일본 1만2793억달러 우리나라의 외환보유액 규모는 2월말 기준 소폭 감소하며 세계 8위 수준(1월)을 유지한 것으로 나타났다.", image_save_path, download_limit=50)

[2019-12-09 07:47:29,797 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.010s]


1.jpg : , Predict : graph
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : others
9.jpg : , Predict : graph
10.jpg : , Predict : others
11.jpg : , Predict : graph
12.jpg : , Predict : graph
13.jpg : , Predict : others
14.jpg : , Predict : others
15.jpg : , Predict : others
16.jpg : , Predict : graph
17.jpg : , Predict : others
18.jpg : , Predict : others
19.jpg : , Predict : others
20.jpg : , Predict : others
21.jpg : , Predict : others
22.jpg : , Predict : others
23.jpg : , Predict : others
24.jpg : , Predict : others
25.jpg : , Predict : others
26.jpg : , Predict : others
27.jpg : , Predict : graph
28.jpg : , Predict : others
29.jpg : , Predict : graph
30.jpg : , Predict : others
31.jpg : , Predict : others
32.jpg : , Predict : others
33.jpg : , Predict : graph
34.jpg : , Predict : graph
35.jpg : , Predict : graph
36.jpg : , Predict : graph
37.jpg : , Predict

In [10]:
image_save_path = "./results/images/우리나라_외환보유액_감소_2.jpg"
image_avail = imageSelector.process_data("한은 관계자는 미 달러화 강세에 따라 기타통화 표시 외화자산의 달러화 환산액이 감소해 지난달 외환보유액이 줄었다 고 설명했다.", image_save_path, download_limit=50)

[2019-12-09 07:47:33,960 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.010s]


1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : others
6.jpg : , Predict : others
7.jpg : , Predict : graph
8.jpg : , Predict : others
9.jpg : , Predict : graph
10.jpg : , Predict : others
11.jpg : , Predict : others
12.jpg : , Predict : others
13.jpg : , Predict : graph
14.jpg : , Predict : others
15.jpg : , Predict : others
16.jpg : , Predict : graph
17.jpg : , Predict : others
18.jpg : , Predict : graph
19.jpg : , Predict : graph
20.jpg : , Predict : others
21.jpg : , Predict : graph
22.jpg : , Predict : others
23.jpg : , Predict : graph
24.jpg : , Predict : graph
25.jpg : , Predict : graph
26.jpg : , Predict : graph
27.jpg : , Predict : graph
28.jpg : , Predict : graph
29.jpg : , Predict : graph
30.jpg : , Predict : graph
31.jpg : , Predict : graph
32.jpg : , Predict : graph
33.jpg : , Predict : graph
34.jpg : , Predict : graph
35.jpg : , Predict : others
36.jpg : , Predict : graph
37.jpg : , Predict : ot

In [48]:
query_body = scouter.make_keyword_query_body("우리나라 외환보유액 감소", match_item='extContent', filters=['analyzed_text', 'postingDate'])
detail_outlook_data = scouter.search(query_body, data_type=default_newspaper_idx, max_num_doc=500, trim_lower=False)

forecast_sentence = forecast.process_data(detail_outlook_data, 5)

detail_json['outlook'] = [sen.split('\t')[0].strip() for sen in forecast_sentence]

Query : 우리나라 외환보유액 감소


[2019-12-09 07:19:52,862 INFO] GET http://155.230.34.145:9200/newspaper/_search?scroll=10m&size=500 [status:200 request:3.545s]


Scroll idx : 1 (500 docs)
Total retrieved Doc # :  500



NameError: name 'detail_json' is not defined

In [50]:
[sen.split('\t')[0].strip() for sen in forecast_sentence]

['반면 자동차, 건설, 철강 산업은 상반기 평가와 하반기 전망 모두 부진할 것으로 평가했다.',
 '김태영 회장은  이번 연수프로그램 등을 통해 해외 국가 은행산업 관계자와의 우호적인 인적 네트워크를 형성하고, 상호 협력의 기틀을 마련함으로써 국내은행의 해외 진출을 효과적으로 지원할 수 있는 계기가 될 것으로 기대한다 고 밝혔다.',
 '주요 산업별로는, 제약 바이오, 정보통신 정유 산업은 올해 상반기에 이어 하반기에도 비교적 양호한 실적을 기록할 것으로 예상했다.',
 '우리나라 단기외채 비율이 올해 들어 더 높아진 것으로 나타났다.',
 '한공회는  우리나라 경제 전반에 대한 BSI 설문조사 결과, 상반기 현황 평가 61, 하반기 전망 64로 나타났는데, 이는 공인회계사들이 경기 부진 국면을 매우 심각하게 평가하고 있음을 보여준다 며  이번 설문조사가 2018년 회계감사가 종료된 직후에 실시된 점을 고려할 때 더욱 엄중히 받아들여야 할 것 이라고 밝혔다.']

In [4]:
%%time
keyword_set_list = [
    ["터키", "리라", "급락"],
    ["임대", "가구", "부채"],
    ["조양호", "연임", "실패"],
#    ["트럼프", "OPEC", "발언"],
    ["재감사", "보수", "증가"],
    ["중국", "트럼프", "관세"],
#    ["우리나라", "외환보유액", "감소"],
    ["SK", "엠플러스", "계약"],
    ["하나은행", "한부모가족", "전세자금대출"],
    ["코스피지수", "하락", "내림새"],
    ["수출", "부진", "경상흑자"],
    ["트럼프", "시진핑", "관세"],
    ["EU", "유로존", "하락"],
    ["주택", "공사", "MBS"],
#    ["브렉시트", "메이", "총리", "英"],
#    ["미국", "중국", "무역"],
    ["중고차", "대출", "제한"],
    ["유니온페이", "해외", "결제"],
    ["한은", "통화신용정책", "보고서"],
    ["삼성자산운용", "연금", "TDF"]
]
for keyword_set in keyword_set_list:
    query_body = scouter.make_keyword_query_body(' '.join(keyword_set), match_item='extContent', filters=['newsTitle', 'extContent', 'originalUrl'])
    detail_summary_data = scouter.search(query_body, data_type="newspaper", max_num_doc=8, trim_lower=False)

    detail_json = {}

    detail_json['print_title_photo'] = None
        
    subkeywords = []
    
    outline_info = {"title":detail_summary_data[0]['newsTitle'],
                    "text":[],
                    "link":[]}

    outline_info['text'].append(multi_doc_summarizer.process_data([doc['extContent'] for doc in detail_summary_data]))
    outline_info['link'].extend([doc['originalUrl'] for doc in detail_summary_data])

    detail_json['outline'] = outline_info
    detail_json['subkeywords'] = subkeywords

    keyword_for_timeline = ' '.join(keyword_set) if ' '.join(keyword_set) != '트럼프 OPEC 발언' else '국제 유가 OPEC'
    query_body = scouter.make_keyword_query_body(keyword_for_timeline, match_item='extContent')
    detail_timeline_data = scouter.search(query_body, data_type=default_timeline_idx, max_num_doc=1000, trim_lower=False)
    detail_timeline_data = [doc for doc in detail_timeline_data if "error" not in doc.keys()]

    summary, _ = timeline_summ.process_data(keyword_for_timeline, detail_timeline_data)
    dates = [sentence['date'] for sentence in summary]
    timeline_sen = [' '.join(sentence['compResult']) for sentence in summary]
    normal_sentence = [sentence['article'][0]['sentence'].strip() for sentence in summary]

    detail_json['timeline'] = []
    for t_idx, (sen, date) in enumerate(zip(normal_sentence, dates)):
        image_save_path = "./results/images/{}_{}.jpg".format('_'.join(keyword_set), t_idx)
        image_avail = imageSelector.process_data(sen, image_save_path, download_limit=50)
    
        timeline_section = {"text":sen,
                            "date":date,
                            "photo":image_save_path if image_avail else None}

        detail_json['timeline'].append(timeline_section)

    query_body = scouter.make_keyword_query_body(keyword_for_timeline, match_item='extContent', filters=['analyzed_text', 'postingDate'])
    detail_outlook_data = scouter.search(query_body, data_type=default_newspaper_idx, max_num_doc=500, trim_lower=False)

    forecast_sentence = forecast.process_data(detail_outlook_data, 5)

    detail_json['outlook'] = [sen.split('\t')[0].strip() for sen in forecast_sentence]

    with open("./results/detail_issue/{}.json".format('_'.join(keyword_set)), "w", encoding='utf8') as fp:
        fp.write(json.dumps(detail_json, ensure_ascii=False))

[2019-12-09 07:49:58,402 INFO] GET http://155.230.34.145:9200/newspaper/_search?scroll=10m&size=8 [status:200 request:0.080s]


Query : 터키 리라 급락
Scroll idx : 1 (8 docs)
Total retrieved Doc # :  8

Query : 터키 리라 급락


[2019-12-09 07:50:09,581 INFO] GET http://155.230.34.145:9200/summary_by_time/_search?scroll=10m&size=1000 [status:200 request:1.383s]


Scroll idx : 1 (1000 docs)
Total retrieved Doc # :  1000

A :  0.022411823272705078
B :  0.6303262710571289
C :  0.021674156188964844
C-sub :  0.04203319549560547
 >> phase 4. calculate next nodes
timeline
['2016-00-00', '2017-00-00', '2018-00-00', '2018-10-25', '2019-02-13', '2019-05-14']
date_idx: 1/6
date_idx: 2/6
date_idx: 3/6
date_idx: 4/6
date_idx: 5/6
 >> phase 4. Make beamsearch file
D :  0.00529170036315918


[2019-12-09 07:50:11,018 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.013s]
[2019-12-09 07:50:11,030 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.010s]
[2019-12-09 07:50:11,039 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.007s]
[2019-12-09 07:50:11,052 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.009s]


...

1.jpg : , Predict : others
2.jpg : , Predict : others
3.jpg : , Predict : graph
4.jpg : , Predict : others
5.jpg : , Predict : others
6.jpg : , Predict : graph
7.jpg : , Predict : others
8.jpg : , Predict : others
9.jpg : , Predict : others
10.jpg : , Predict : graph
11.jpg : , Predict : graph
12.jpg : , Predict : graph
13.jpg : , Predict : graph
14.jpg : , Predict : graph
15.jpg : , Predict : graph
16.jpg : , Predict : others
17.jpg : , Predict : graph
18.jpg : , Predict : others
19.jpg : , Predict : graph
20.jpg : , Predict : others
21.jpg : , Predict : graph
22.jpg : , Predict : others
23.jpg : , Predict : graph
24.jpg : , Predict : graph
25.jpg : , Predict : graph
26.jpg : , Predict : graph
27.jpg : , Predict : graph
28.jpg : , Predict : others
29.jpg : , Predict : graph
30.jpg : , Predict : graph
31.jpg : , Predict : graph
32.jpg : , Predict : graph
33.jpg : , Predict : graph
34.jpg : , Predict : graph
35.jpg : , Predict : others
36.jpg : , Predict : others
37.jpg : , Predict : ot

[2019-12-09 08:28:29,223 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.010s]


1.jpg : , Predict : graph
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : graph
10.jpg : , Predict : graph
11.jpg : , Predict : graph
12.jpg : , Predict : graph
13.jpg : , Predict : graph
14.jpg : , Predict : graph
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : graph
18.jpg : , Predict : others
19.jpg : , Predict : graph
20.jpg : , Predict : others
21.jpg : , Predict : others
22.jpg : , Predict : graph
23.jpg : , Predict : graph
24.jpg : , Predict : others
25.jpg : , Predict : others
26.jpg : , Predict : graph
27.jpg : , Predict : graph
28.jpg : , Predict : graph
29.jpg : , Predict : others
30.jpg : , Predict : graph
31.jpg : , Predict : others
32.jpg : , Predict : graph
33.jpg : , Predict : graph
34.jpg : , Predict : graph
35.jpg : , Predict : graph
36.jpg : , Predict : others
37.jpg : , Predict : graph
38

[2019-12-09 08:28:37,646 INFO] GET http://155.230.34.145:9200/newspaper/_search?scroll=10m&size=500 [status:200 request:4.298s]


Scroll idx : 1 (500 docs)
Total retrieved Doc # :  500

CPU times: user 39min 6s, sys: 2min 54s, total: 42min
Wall time: 39min 6s


In [14]:
query_body = scouter.make_keyword_query_body('우리나라 외환보유액 감소', match_item='extContent', filters=['analyzed_text', 'postingDate'])
detail_outlook_data = scouter.search(query_body, data_type=default_newspaper_idx, max_num_doc=1000, trim_lower=False)

Query : 우리나라 외환보유액 감소


[2019-12-06 08:39:25,558 INFO] GET http://155.230.34.145:9200/newspaper/_search?scroll=10m&size=1000 [status:200 request:5.617s]


Scroll idx : 1 (1000 docs)
Total retrieved Doc # :  1000



In [15]:
forecast_sentence = forecast.process_data(detail_outlook_data, "2019-01-01")
[sen.split('\t')[0].strip() for sen in forecast_sentence]

['미중 무역분쟁과 미 연준의 추가 금리인상 등 불안 요소가 이어지면서, 우리 증시는 내년에도 살얼음판을 걸을 전망입니다.',
 '연구진은  글로벌 금융위기 이후 저금리 기간 중에는 우리나라 통화 정책이 자본유출입에 미치는 영향이 약화된 것으로 추정된다 며  다만 통화정책의 완화정도가 축소되면 자본이동에 미치는 영향이 확대될 수 있어 유의해야 한다 고 덧붙였다.',
 '연구진은  글로벌 금융위기 이후 저금리 기간 중에는 우리나라 통화 정책이 자본유출입에 미치는 영향이 약화된 것으로 추정된다 며  다만 통화정책의 완화정도가 축소되면 자본이동에 미치는 영향이 확대될 수 있어 유의해야 한다 고 덧붙였다.',
 '연구진은  소규모 개방경제인 우리나라 통화정책이 자본유출입에 미치는 영향을 분석한 결과, 글로벌 금융위기 이후 저금리 기간 중 우리나라 통화정책이 자본유출입에 미치는 영향이 약화된 것으로 추정된다 며  다만 통화정책의 완화정도가 축소되는 경우 통화정책이 자본이동에미치는 영향이 확대될 수 있음에 유의해야한다 고 밝혔다.',
 '다만 금리가 1%대 수준에서는 정책금리 수준이 낮아질수록 통화정책이 자본 유출에 미치는 영향이 상당한 것으로 드러났다.']

[]

In [7]:
image_save_path = "b.jpg"
imageSelector.process_data("미중 무역전쟁", image_save_path, download_limit=50)

[2019-12-06 05:42:47,646 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.007s]


query is new
query pre-processed


[2019-12-06 05:42:58,635 INFO] POST http://155.230.34.145:9200/image/_doc [status:201 request:0.079s]


(50, 224, 224, 3)
vgg predict - success
1.jpg : , Predict : graph
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : graph
10.jpg : , Predict : others
11.jpg : , Predict : others
12.jpg : , Predict : graph
13.jpg : , Predict : graph
14.jpg : , Predict : graph
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : graph
18.jpg : , Predict : graph
19.jpg : , Predict : graph
20.jpg : , Predict : graph
21.jpg : , Predict : graph
22.jpg : , Predict : graph
23.jpg : , Predict : graph
24.jpg : , Predict : graph
25.jpg : , Predict : graph
26.jpg : , Predict : graph
27.jpg : , Predict : graph
28.jpg : , Predict : graph
29.jpg : , Predict : graph
30.jpg : , Predict : graph
31.jpg : , Predict : graph
32.jpg : , Predict : graph
33.jpg : , Predict : graph
34.jpg : , Predict : others
35.jpg : , Predict : graph
36.jpg : , Predict : 

In [26]:
summary, _ = timeline_summ.process_data(keyword_for_timeline, detail_timeline_data)
dates = [sentence['date'] for sentence in summary]
timeline_sen = [' '.join(sentence['compResult']) for sentence in summary]
normal_sentence = [sentence['article'][0]['sentence'].strip() for sentence in summary]

A :  0.016620159149169922
B :  0.44138169288635254
C :  0.011210441589355469
C-sub :  0.010244607925415039
 >> phase 4. calculate next nodes
timeline
['2018-00-00', '2019-00-00', '2019-02-26', '2019-04-23']
date_idx: 1/4
date_idx: 2/4
date_idx: 3/4
 >> phase 4. Make beamsearch file
D :  0.0031540393829345703
E :  0.21872973442077637


In [19]:
detail_json

{'outline': {'title': '미국, 중국과 무역협상 시한 60일 연장 검토',
  'text': ['므누신 미 재무장관과 로버트 라이트하이저 미 무역대표부(ustr) 대표가 베이징에서 재차 고위급 무역협상 시한을 60일 늦추는 방안을 검토하고 있다고 블룸버그 통신이 보도했다. 시진핑 중국 국가주석이 무역 협상 타결을 위해 관세 인상 시한인 3월 1일을 60일 더 연장할 수 있음을 시사했다. ',
   '도널드 트럼프 미 대통령이 중국산 수입품에 대한 관세 철회 가능성을 시사했다. 트럼프는 시진핑 중국 국가주석과 로버트 라이트하이저 미국 무역대표부(ustr) 대표의 발언에 대해 논의하고 있다고 월스트리트저널(wsj)이 1일(현지시간) 보도했다. 미-중 무역협상 타결은 미중 무역 분쟁이 해결될 수 있다고 경고했다',
   '도널드 트럼프 미 대통령이 10일(현지시간) 워싱턴dc에서 2,000억 달러 상당의 중국산 수입품에 25% 관세를 부과할 계획이라고 밝혔다. 트럼프는 시진핑 중국 국가주석과 류허 부총리, 스티븐 므누신 미 재무장관이 보도했다. 양국 고위급 무역대표부(ustr) 대표가 관세율을 10%에서 25%로 상향 조정했다. '],
  'link': ['https://www.sedaily.com/NewsView/1VFB7I93VH',
   'http://www.fnnews.com/news/201902141355230993',
   'http://www.hani.co.kr/arti/international/globaleconomy/882154.html',
   'http://www.edaily.co.kr/news/newspath.asp?newsid=03909766622390520',
   'http://news1.kr/articles/?3547777',
   'http://news.mt.co.kr/mtview.php?no=2019032107273231859',
   'http://www.newsis.com/view/?id=NISX20190321_0

Query : 국제 유가 OPEC


[2019-12-02 06:45:44,508 INFO] GET http://155.230.34.145:9200/summary_by_time/_search?scroll=10m&size=1000 [status:200 request:0.905s]
[2019-12-02 06:45:45,064 INFO] GET http://155.230.34.145:9200/_search/scroll?scroll=10m [status:200 request:0.012s]


Scroll idx : 1 (542 docs)
Total retrieved Doc # :  542

A :  0.016189098358154297
B :  0.4231894016265869
C :  0.012908935546875
C-sub :  0.026672840118408203
 >> phase 4. calculate next nodes
timeline
['2018-00-00', '2019-00-00', '2019-02-26', '2019-04-23']
date_idx: 1/4
date_idx: 2/4
date_idx: 3/4
 >> phase 4. Make beamsearch file
D :  0.004630088806152344


[2019-12-02 06:45:45,780 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.013s]


E :  0.22072172164916992
query is new
query pre-processed
query already cached in scouter
(50, 224, 224, 3)
vgg predict - success
1.jpg : , Predict : graph
2.jpg : , Predict : graph
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : graph
10.jpg : , Predict : graph
11.jpg : , Predict : graph
12.jpg : , Predict : graph
13.jpg : , Predict : graph
14.jpg : , Predict : graph
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : graph
18.jpg : , Predict : others
19.jpg : , Predict : graph
20.jpg : , Predict : others
21.jpg : , Predict : graph
22.jpg : , Predict : graph
23.jpg : , Predict : others
24.jpg : , Predict : graph
25.jpg : , Predict : graph
26.jpg : , Predict : others
27.jpg : , Predict : others
28.jpg : , Predict : graph
29.jpg : , Predict : graph
30.jpg : , Predict : graph
31.jpg : , Predict : graph
32.jpg : , Predict : graph
33.jpg : ,

...

query already used before
['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '46.jpg', '47.jpg', '48.jpg', '49.jpg', '50.jpg']
['./downloads/트럼프 대통령은  트위터에서 유가가 지나치게 높아졌다 며 OPEC은 제발 진정해라./1.jpg', './downloads/트럼프 대통령은  트위터에서 유가가 지나치게 높아졌다 며 OPEC은 제발 진정해라./2.jpg', './downloads/트럼프 대통령은  트위터에서 유가가 지나치게 높아졌다 며 OPEC은 제발 진정해라./3.jpg', './downloads/트럼프 대통령은  트위터에서 유가가 지나치게 높아졌다 며 OPEC은 제발 진정해라./4.jpg', './downloads/트럼프 대통령은  트위터에서 유가가 지나치게 높아졌다 며 OPEC은 제발 진정해라./5.jpg', './downloads/트럼프 대통령은  트위터에서 유가가 지나치게 높아졌다 며 OPEC은 제발 진정해라./6.jpg', './downloads/트럼프 대통령은  트위터에서 유가가 지나치게 높아졌다 며 OPEC은 제발 진정해라./7.jpg', './do

[2019-12-02 06:46:10,050 INFO] GET http://155.230.34.145:9200/image/_search [status:200 request:0.007s]


query is new
query pre-processed


[2019-12-02 06:46:21,504 INFO] POST http://155.230.34.145:9200/image/_doc [status:201 request:0.048s]


(50, 224, 224, 3)
vgg predict - success
1.jpg : , Predict : graph
2.jpg : , Predict : others
3.jpg : , Predict : graph
4.jpg : , Predict : graph
5.jpg : , Predict : graph
6.jpg : , Predict : graph
7.jpg : , Predict : graph
8.jpg : , Predict : graph
9.jpg : , Predict : graph
10.jpg : , Predict : graph
11.jpg : , Predict : graph
12.jpg : , Predict : graph
13.jpg : , Predict : graph
14.jpg : , Predict : graph
15.jpg : , Predict : graph
16.jpg : , Predict : graph
17.jpg : , Predict : graph
18.jpg : , Predict : graph
19.jpg : , Predict : graph
20.jpg : , Predict : graph
21.jpg : , Predict : graph
22.jpg : , Predict : graph
23.jpg : , Predict : graph
24.jpg : , Predict : others
25.jpg : , Predict : graph
26.jpg : , Predict : graph
27.jpg : , Predict : graph
28.jpg : , Predict : graph
29.jpg : , Predict : graph
30.jpg : , Predict : graph
31.jpg : , Predict : graph
32.jpg : , Predict : graph
33.jpg : , Predict : graph
34.jpg : , Predict : graph
35.jpg : , Predict : graph
36.jpg : , Predict : g

In [47]:
%%

Query : 국제 유가 OPEC


[2019-12-02 06:51:38,869 INFO] GET http://155.230.34.145:9200/newspaper/_search?scroll=10m&size=100 [status:200 request:0.959s]


Scroll idx : 1 (100 docs)
Total retrieved Doc # :  100

CPU times: user 14.2 s, sys: 128 ms, total: 14.4 s
Wall time: 6.19 s


In [49]:
detail_json['related'] = ['대한민국 대한민국', "중국 중국 중국", "미국 미국 미국", "일본 일본 일본 일본"]

In [54]:
with open("./results/detail_issue/{}.json".format('_'.join(keyword_set)), "w", encoding='utf8') as fp:
    fp.write(json.dumps(detail_json, ensure_ascii=False))

In [52]:
keyword_set

'트럼프OPEC발언'

In [16]:
#query_body = scouter.make_keyword_query_body('트럼프 OPEC 발언', match_item='extContent', filters=['analyzed_text', 'postingDate'])
#detail_outlook_data = scouter.search(query_body, data_type=default_newspaper_idx, max_num_doc=1000, trim_lower=False)
#
#sentences = forecast.process_data(detail_outlook_data, date)

Query : 트럼프 OPEC 발언


[2019-11-29 08:11:20,620 INFO] GET http://155.230.34.145:9200/newspaper/_search?scroll=10m&size=1000 [status:200 request:8.169s]


Scroll idx : 1 (1000 docs)
Total retrieved Doc # :  1000



/workspace/wise_reporter/modules/forecast/prediction.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = a(output)
